### Задачи
1.	Выполните сохранение монохромного изображения в виде текстового или бинарного файла.
2.	Реализуйте алгоритм вейвлет-преобразования Хаара для изображения.
3.	Выполните квантование высокочастотных компонент (прим., количество квантов  = 4).
4.	Сохраните получившийся массив значений  в текстовый или бинарный файл в порядке LL, LH, HL, HH вейвлет-преобразования Хафа. Компоненты LH, HL, HH храните в виде пар (значение, количество повторений).
Сравните объем памяти, занимаемый исходным изображением (попиксельное хранение), и изображение, полученным после преобразования Хафа и сжатием длин серий.


In [36]:
import numpy as np
import cv2 as cv
import matplotlib as plt

np.seterr(all='ignore')

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [37]:
def save_image_txt(image, filename):
    image = image.astype(int)
    np.savetxt(filename, image, fmt='%d')

def load_image_txt(filename):
    return np.loadtxt(filename)

In [38]:
def haar_transform_2d(image):
    rows, cols = image.shape

    for i in range(rows):
        row = image[i, :].copy()
        for j in range(cols // 2):
            image[i, j] = (row[2*j] + row[2*j+1]) / 2
            image[i, cols//2 + j] = (row[2*j] - row[2*j+1]) / 2

    for j in range(cols):
        col = image[:, j].copy()
        for i in range(rows // 2):
            image[i, j] = (col[2*i] + col[2*i+1]) / 2
            image[rows//2 + i, j] = (col[2*i] - col[2*i+1]) / 2
    return image

In [39]:
def quantize(component, levels=4):
    min_val, max_val = np.min(component), np.max(component)
    step = (max_val - min_val) / levels
    quantized = np.floor((component - min_val) / step)
    return np.clip(quantized, 0, levels-1).astype(int)

In [40]:
def rle_encode(arr):
    flat = arr.flatten()
    changes = np.where(flat[1:] != flat[:-1])[0] + 1
    starts = np.concatenate(([0], changes))
    stops = np.concatenate((changes, [len(flat)]))
    return list(zip(flat[starts], stops - starts))

In [41]:
def save_compressed_txt(ll, lh, hl, hh, filename):
    with open(filename, 'w') as f:
        f.write(f"LL {ll.shape[0]} {ll.shape[1]}\n")
        np.savetxt(f, ll, fmt='%.2f')
        
        for comp, name in [(lh, 'LH'), (hl, 'HL'), (hh, 'HH')]:
            encoded = rle_encode(comp)
            f.write(f"{name} {len(encoded)}\n")
            for val, count in encoded:
                f.write(f"{val} {count}\n")

In [42]:
filename = "vosmerka"
img = cv.imread(f"{filename}.jpg", cv.IMREAD_GRAYSCALE).astype(np.float16)
save_image_txt(img, f"{filename}.txt")


transformed = haar_transform_2d(img.copy())

rows, cols = transformed.shape
LL = transformed[:rows//2, :cols//2]
LH = transformed[:rows//2, cols//2:]
HL = transformed[rows//2:, :cols//2]
HH = transformed[rows//2:, cols//2:]

LH_q = quantize(LH)
HL_q = quantize(HL)
HH_q = quantize(HH)

save_compressed_txt(LL, LH_q, HL_q, HH_q, f"compressed_{filename}.txt")

with open(f'{filename}.txt', 'r') as f:
    original_size = len(f.read().encode('utf-8'))

with open(f'compressed_{filename}.txt', 'r') as f:
    compressed_size = len(f.read().encode('utf-8'))

print(f"Original size: {original_size} bytes")
print(f"Compressed size: {compressed_size} bytes")
print(f"Compression ratio: {original_size / compressed_size:.2f}")

Original size: 1780073 bytes
Compressed size: 1248558 bytes
Compression ratio: 1.43
